# Assignment 4

## Description

In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Dependencies

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [8]:
def question_one():

    nhl_df = pd.read_csv('assets/nhl.csv')
    cities = pd.read_html('assets/wikipedia_data.html')[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    def clear_string(string: str) -> str:
        regex = re.search(r'\[[a-z]* [0-9]+\]', string)
        if regex is None:
            return string
        else:
            return string.replace(regex.group(), '')


    def get_area(team):
        for each in list(nhl_cities.index.values):
            if team in each:
                return nhl_cities.at[each, 'Metropolitan area']
        

    cities['NHL'] = cities['NHL'].apply(lambda x: clear_string(x))
    nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
    nhl_cities.drop(['—', ''], axis=0, inplace=True)

    nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0)
    
    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
    population.set_index('Metropolitan area', inplace=True)

    nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith('*') else x.strip())
    nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(' ')[-1])
    nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))

    res = []
    for group, frame in nhl_df.groupby('area'):
        wins = frame['W'].astype(int).sum()
        loss = frame['L'].astype(int).sum()
        total = wins + loss
        ratio = wins / total
        output = {'area': group, 'ratio': ratio}
        res.append(output)

    res = pd.DataFrame(res)
    res.set_index('area', inplace=True)
    out = pd.merge(res, population, how='inner', left_index=True, right_index=True)
    out['Population (2016 est.)[8]'] = pd.to_numeric(out['Population (2016 est.)[8]'], downcast='integer')
    
    return out
    

question_one()

,ratio,Population (2016 est.)[8]
Boston,0.714286,4794447
Buffalo,0.357143,1132804
Calgary,0.513889,1392609
Chicago,0.458333,9512999
Columbus,0.600000,2041520
Dallas–Fort Worth,0.567568,7233323
Denver,0.589041,2853077
Detroit,0.434783,4297617
Edmonton,0.473684,1321426
Las Vegas,0.680000,2155664


In [9]:
def nhl_correlation(): 

    population_by_region = question_one()['Population (2016 est.)[8]'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = question_one()['ratio'].to_list() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [10]:
nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [11]:
def question_two():

    nba_df = pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    # Helper functions
    def clear_note(string: str) -> str: 
        regex = re.search(r'\[[a-z]* [0-9]*\]', string)
        if regex is None:
            return string
        else:
            return string.replace(regex.group(), '')


    def clear_team(string: str) -> list:
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', str(string))).split()
        return splitted


    # Clean cities dataframe
    cities.drop(['NFL', 'MLB', 'NHL'], axis=1, inplace=True)
    cities['NBA'] = cities['NBA'].apply(lambda x: clear_note(x))
    cities['NBA'] = cities['NBA'].replace('', np.nan)
    cities['NBA'] = cities['NBA'].replace('—', np.nan)
    cities['NBA'] = cities['NBA'].apply(lambda x: clear_team(x))
    # Drop all rows with cities that are not in the list
    cities = cities.explode('NBA')
    cities.dropna(inplace=True)
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)

    # Clean nba dataframe
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df.drop(['League', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS'], axis=1, inplace=True)
    nba_df['team'] = nba_df['team'].replace(r'\([0-9]*\)', '', regex=True)
    nba_df['team'] = nba_df['team'].replace(r'\*', '', regex=True)

    nba_df.W = nba_df.W.astype(int)
    nba_df.L = nba_df.L.astype(int)
    nba_df['WL'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])

    temp_dic = {'Toronto Raptors':'Toronto',
                'Boston Celtics':'Boston',
                'Philadelphia 76ers':'Philadelphia',
                'Cleveland Cavaliers':'Cleveland',
                'Indiana Pacers':'Indianapolis',
                'Miami Heat':'Miami–Fort Lauderdale',
                'Milwaukee Bucks':'Milwaukee',
                'Washington Wizards':'Washington, D.C.',
                'Detroit Pistons':'Detroit',
                'Charlotte Hornets':'Charlotte',
                'New York Knicks':'New York City',
                'Brooklyn Nets':'New York City',
                'Chicago Bulls':'Chicago',
                'Orlando Magic':'Orlando',
                'Atlanta Hawks':'Atlanta',
                'Houston Rockets':'Houston',
                'Golden State Warriors':'San Francisco Bay Area',
                'Portland Trail Blazers':'Portland',
                'Oklahoma City Thunder':'Oklahoma City',
                'Utah Jazz':'Salt Lake City',
                'New Orleans Pelicans':'New Orleans',
                'San Antonio Spurs':'San Antonio',
                'Minnesota Timberwolves':'Minneapolis–Saint Paul',
                'Denver Nuggets':'Denver',
                'Los Angeles Clippers':'Los Angeles',
                'Los Angeles Lakers':'Los Angeles',
                'Sacramento Kings':'Sacramento',
                'Dallas Mavericks':'Dallas–Fort Worth',
                'Memphis Grizzlies':'Memphis',
                'Phoenix Suns':'Phoenix'}

    nba_df['team'] = nba_df['team'].apply(lambda x: x.strip())
    nba_df['Metropolitan area'] = nba_df['team'].replace(temp_dic)

    # Merge nba and cities
    temp = pd.merge(nba_df, cities, how='inner', left_on='Metropolitan area', right_on='Metropolitan area')
    # Remove duplicate rows where team is the same
    temp.drop_duplicates(subset=['team'], inplace=True)
    temp.drop(['W', 'L', 'NBA', 'year', 'team'], axis=1, inplace=True)

    # print rows where metropolitan area is repeated
    # Replace repeated rows with their average


    return temp.groupby('Metropolitan area').mean()

question_two()

,WL,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.292683,5789700.0
Boston,0.670732,4794447.0
Charlotte,0.439024,2474314.0
Chicago,0.329268,9512999.0
Cleveland,0.609756,2055612.0
Dallas–Fort Worth,0.292683,7233323.0
Denver,0.560976,2853077.0
Detroit,0.475610,4297617.0
Houston,0.792683,6772470.0


In [12]:
def nba_correlation():
    # YOUR CODE HERE
    res = question_two()
    
    population_by_region = res['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = res['WL'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [13]:
nba_correlation()


(-0.17657160252844617, 0.36874741604463)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [14]:
def question_three():
    
    mlb_df = pd.read_csv("assets/mlb.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    
    def clear_note(string: str) -> str: 
        regex = re.search(r'\[[a-z]* [0-9]*\]', string)
        if regex is None:
            return string
        else:
            return string.replace(regex.group(), '')


    def clear_team(string: str) -> list:
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', str(string).split()[0])).split()
        return splitted

    # Clean cities dataframe
    cities.drop(['NFL', 'NHL', 'NBA'], axis=1, inplace=True)
    cities['MLB'] = cities['MLB'].apply(lambda x: clear_note(x))
    cities['MLB'] = cities['MLB'].replace('', np.nan)
    cities['MLB'] = cities['MLB'].replace('—', np.nan)
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    cities['MLB'] = cities['MLB'].apply(lambda x: clear_team(x))
    cities = cities.explode('MLB')
    cities = cities[cities['MLB'] != 'nan']

    # Clean mlb dataframe
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df.drop(['W-L%', 'GB', 'year', 'League'], inplace=True, axis=1)
    mlb_df['WL'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])

    temp_dict = {'Boston Red Sox': 'Boston',
                'New York Yankees': 'New York City', 
                'Tampa Bay Rays': 'Tampa Bay Area',
                'Toronto Blue Jays': 'Toronto',
                'Baltimore Orioles': 'Baltimore', 
                'Cleveland Indians': 'Cleveland',
                'Minnesota Twins': 'Minneapolis–Saint Paul', 
                'Detroit Tigers': 'Detroit', 
                'Chicago White Sox': 'Chicago',
                'Kansas City Royals': 'Kansas City', 
                'Houston Astros': 'Houston', 
                'Oakland Athletics': 'San Francisco Bay Area',
                'Seattle Mariners' : 'Seattle', 
                'Los Angeles Angels': 'Los Angeles', 
                'Texas Rangers': 'Dallas–Fort Worth',
                'Atlanta Braves': 'Atlanta', 
                'Washington Nationals': 'Washington, D.C.', 
                'Philadelphia Phillies': 'Philadelphia',
                'New York Mets': 'New York City',
                'Miami Marlins': 'Miami–Fort Lauderdale',
                'Milwaukee Brewers': 'Milwaukee', 
                'Chicago Cubs': 'Chicago',
                'St. Louis Cardinals': 'St. Louis', 
                'Pittsburgh Pirates': 'Pittsburgh', 
                'Cincinnati Reds': 'Cincinnati',
                'Los Angeles Dodgers': 'Los Angeles', 
                'Colorado Rockies': 'Denver', 
                'Arizona Diamondbacks': 'Phoenix',
                'San Francisco Giants': 'San Francisco Bay Area', 
                'San Diego Padres': 'San Diego'}

    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip())

    mlb_df['Metropolitan area'] = mlb_df['team'].replace(temp_dict)

    temp = pd.merge(mlb_df, cities, how='inner', left_on='Metropolitan area', right_on='Metropolitan area')
    temp.drop_duplicates(subset=['team'], inplace=True)
    temp.drop(['W', 'L', 'MLB', 'team'], axis=1, inplace=True)

    return temp.groupby('Metropolitan area').mean()


question_three()

,WL,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.555556,5789700.0
Baltimore,0.290123,2798886.0
Boston,0.666667,4794447.0
Chicago,0.482769,9512999.0
Cincinnati,0.413580,2165139.0
Cleveland,0.561728,2055612.0
Dallas–Fort Worth,0.413580,7233323.0
Denver,0.558282,2853077.0
Detroit,0.395062,4297617.0


In [15]:
def mlb_correlation(): 
    # YOUR CODE HERE
    res = question_three()
    
    population_by_region = res['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = res['WL'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [16]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [17]:
def question_four():

    nfl_df = pd.read_csv("assets/nfl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # Helper functions
    def clear_note(string: str) -> str: 
        regex = re.search(r'\[[a-z]* [0-9]*\]', string)
        if regex is None:
            return string
        else:
            return string.replace(regex.group(), '')


    def clear_team(string: str) -> list:
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', str(string).split()[0])).split()
        return splitted

    # Clean cities dataframe
    cities.drop(['MLB', 'NHL', 'NBA'], axis=1, inplace=True)
    cities['NFL'] = cities['NFL'].apply(lambda x: clear_note(x))
    cities['NFL'] = cities['NFL'].apply(lambda x: x.strip())
    cities['NFL'] = cities['NFL'].replace('', np.nan)
    cities['NFL'] = cities['NFL'].replace('—', np.nan)
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    cities['NFL'] = cities['NFL'].apply(lambda x: clear_team(x))
    cities = cities.explode('NFL')
    cities = cities[cities['NFL'] != 'nan']

    # Clean nfl dataframe
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df.drop(['W-L%', 'year', 'League', 'MoV', 'OSRS', 
                'PA', 'PD', 'PF', 'SRS', 'SoS', 'DSRS', 'T'], inplace=True, axis=1)
    nfl_df['L'] = pd.to_numeric(nfl_df['L'], errors='coerce')
    nfl_df['W'] = pd.to_numeric(nfl_df['W'], errors='coerce')
    nfl_df['WL'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    # Removing * and + signs
    nfl_df['team'] = nfl_df['team'].replace({'\*': ''}, regex=True)
    nfl_df['team'] = nfl_df['team'].replace({'\+': ''}, regex=True)
    nfl_df.dropna(inplace=True)

    temp_dict = {'New England Patriots': 'Boston', 
                'Miami Dolphins': 'Miami–Fort Lauderdale', 
                'Buffalo Bills': 'Buffalo',
                'New York Jets': 'New York City', 
                'Baltimore Ravens': 'Baltimore', 
                'Pittsburgh Steelers': 'Pittsburgh',
                'Cleveland Browns': 'Cleveland', 
                'Cincinnati Bengals': 'Cincinnati', 
                'Houston Texans': 'Houston',
                'Indianapolis Colts': 'Indianapolis', 
                'Tennessee Titans': 'Nashville', 
                'Jacksonville Jaguars': 'Jacksonville',
                'Kansas City Chiefs': 'Kansas City', 
                'Los Angeles Chargers': 'Los Angeles', 
                'Denver Broncos': 'Denver',
                'Oakland Raiders': 'San Francisco Bay Area', 
                'Dallas Cowboys': 'Dallas–Fort Worth', 
                'Philadelphia Eagles': 'Philadelphia',
                'Washington Redskins': 'Washington, D.C.', 
                'New York Giants': 'New York City', 
                'Chicago Bears': 'Chicago',
                'Minnesota Vikings': 'Minneapolis–Saint Paul', 
                'Green Bay Packers': 'Green Bay', 
                'Detroit Lions': 'Detroit',
                'New Orleans Saints': 'New Orleans', 
                'Carolina Panthers': 'Charlotte', 
                'Atlanta Falcons': 'Atlanta',
                'Tampa Bay Buccaneers': 'Tampa Bay Area', 
                'Los Angeles Rams': 'Los Angeles', 
                'Seattle Seahawks': 'Seattle',
                'San Francisco 49ers': 'San Francisco Bay Area', 
                'Arizona Cardinals': 'Phoenix'}

    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.strip())
    nfl_df['Metropolitan area'] = nfl_df['team'].replace(temp_dict)

    temp = pd.merge(nfl_df, cities, how='inner', left_on='Metropolitan area', right_on='Metropolitan area')
    temp.drop_duplicates(subset=['team'], inplace=True)
    temp.drop(['W', 'L', 'NFL', 'team'], axis=1, inplace=True)

    return temp.groupby('Metropolitan area').mean()

question_four()


,WL,Population (2016 est.)[8]
Metropolitan area,,
Atlanta,0.437500,5789700.0
Baltimore,0.625000,2798886.0
Boston,0.687500,4794447.0
Buffalo,0.375000,1132804.0
Charlotte,0.437500,2474314.0
Chicago,0.750000,9512999.0
Cincinnati,0.375000,2165139.0
Cleveland,0.466667,2055612.0
Dallas–Fort Worth,0.625000,7233323.0


In [18]:
def nfl_correlation(): 
    
    res = question_four()
    
    population_by_region = res['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = res['WL'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [19]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [20]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values